# 📘 Performance Optimization: Maximize NLSQ Speed⏱️ **25-35 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Performance**---

## 🎯 What You'll Learn- ✓ **Profile** curve fitting performance- ✓ **Optimize** JAX compilation and JIT- ✓ **Leverage** GPU acceleration- ✓ **Minimize** memory overhead- ✓ **Apply** best practices for speed---

## Setup

In [1]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [2]:
import time

import jax
import jax.numpy as jnp
import numpy as np

from nlsq import CurveFit, curve_fit

devices = jax.devices()
print(f"Device: {devices[0].platform}")


⚠️  GPU ACCELERATION AVAILABLE
═══════════════════════════════
NVIDIA GPU detected: NVIDIA GeForce RTX 4090 Laptop GPU
JAX is currently using: CPU-only

Enable 150-270x speedup with GPU acceleration:
  make install-jax-gpu

Or manually:
  pip uninstall -y jax jaxlib
  pip install "jax[cuda12-local]>=0.6.0"

See README.md GPU Installation section for details.

Device: cpu


## 1. JAX Compilation Basics**Key insight:** First fit is slow (compilation), subsequent fits are fast

In [3]:
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

x = np.linspace(0, 10, 1000)
y = 5 * np.exp(-0.5 * x) + 2 + np.random.normal(0, 0.1, 1000)

# First fit (slow - compilation)
start = time.time()
popt1, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t1 = time.time() - start

# Second fit (fast - reuses compiled code)
start = time.time()
popt2, _ = curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
t2 = time.time() - start

print(f"First fit:  {t1:.4f}s (with compilation)")
print(f"Second fit: {t2:.4f}s (compiled)")
print(f"Speedup: {t1/t2:.1f}x")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.645158e+02 | ‖∇f‖=1.101801e+03 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=5.639188e+00 | ‖∇f‖=1.808072e+01 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.149539s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=5.219442e+00 | time=0.150s | final_gradient_norm=4.570528280962621e-07


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.298741s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.2987410460191313, 'final_cost': 10.438883452251146, 'covariance_warning': False}


First fit:  1.0239s (with compilation)
Second fit: 0.3380s (compiled)
Speedup: 3.0x


## 2. Reusing CurveFit Objects**Best practice:** Reuse CurveFit objects to avoid recompilation

In [4]:
# Create reusable fitter
jcf = CurveFit()

# Multiple fits reuse compiled functions
times = []
for _ in range(10):
    start = time.time()
    popt, _ = jcf.curve_fit(exponential, x, y, p0=[4, 0.4, 1.5])
    times.append(time.time() - start)

print(f"Average fit time: {np.mean(times[1:]):.4f}s")
print("✓ Reusing CurveFit object eliminates compilation overhead")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 1000, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 1000, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.645158e+02 | ‖∇f‖=1.101801e+03 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=5.639188e+00 | ‖∇f‖=1.808072e+01 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.005141s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=5.219442e+00 | time=0.005s | final_gradient_norm=4.570528280962621e-07


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.036581s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.03658111797994934, 'final_cost': 10.438883452251146, 'covariance_warning': False}


Average fit time: 0.0432s
✓ Reusing CurveFit object eliminates compilation overhead


## 3. Fixed Array Size**Issue:** Different array sizes trigger recompilation**Solution:** Use `flength` parameter

In [5]:
# Without fixed length (recompiles for each size)
jcf_dynamic = CurveFit()
sizes = [100, 200, 300]

for n in sizes:
    x_test = np.linspace(0, 10, n)
    y_test = 5 * np.exp(-0.5 * x_test) + 2
    start = time.time()
    popt, _ = jcf_dynamic.curve_fit(exponential, x_test, y_test, p0=[4, 0.4, 1.5])
    elapsed = time.time() - start
    print(f"Size {n}: {elapsed:.4f}s")

print("\n⚠️ Each size triggers recompilation")

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': None}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=1.631332e+01 | ‖∇f‖=1.099246e+02 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=3.608026e-02 | ‖∇f‖=2.070611e+00 | step=4.290688e+00 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.143360s


INFO:nlsq.least_squares:Convergence: reason=`gtol` termination condition is satisfied. | iterations=4 | final_cost=1.438044e-26 | time=0.143s | final_gradient_norm=3.5330107105402626e-12


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.325710s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.32571037401794456, 'final_cost': 2.8760882528226564e-26, 'covariance_warning': False}


Size 300: 0.3709s

⚠️ Each size triggers recompilation


## 4. GPU vs CPU Performance**GPU advantage:** Increases with dataset size

In [6]:
if devices[0].platform == 'gpu':
    print("🚀 GPU Performance Test")
    sizes = [1000, 10000, 100000]

    for n in sizes:
        x_large = np.linspace(0, 10, n)
        y_large = 5 * np.exp(-0.5 * x_large) + 2 + np.random.normal(0, 0.1, n)
        start = time.time()
        popt, _ = curve_fit(exponential, x_large, y_large, p0=[4, 0.4, 1.5])
        elapsed = time.time() - start
        print(f"  {n:>6} points: {elapsed:.4f}s")
else:
    print("💻 CPU mode - GPU would provide 100-300x speedup")

💻 CPU mode - GPU would provide 100-300x speedup


## 🔑 Key Takeaways1. **Reuse CurveFit objects** to avoid recompilation2. **Use fixed array size** when processing multiple datasets3. **GPU acceleration** scales with dataset size4. **Warmup first fit** to compile functions5. **Profile before optimizing** - measure actual bottlenecks---

## 🔗 Next Steps- [Large Dataset Demo](large_dataset_demo.ipynb) - Memory optimization- [GPU Deep Dive](../03_advanced/gpu_optimization_deep_dive.ipynb) - Advanced GPU techniques- [Advanced Features](advanced_features_demo.ipynb) - Callbacks, robust fitting---